# Generating notes for a midi file based on the trained network

In [1]:
import tensorflow.compat.v1 as tf
from tensorflow.keras import datasets, layers, models, optimizers
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

import pickle
import numpy as np

from music21 import converter, instrument, note, chord, stream

from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization as BatchNorm

### Data Preparation

In [2]:
# Loading in the notes from the dataset which have been parsed through music21
# Predefining parameters which will be called upon

In [3]:
def generate():
    
    # load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

### Data Preprocessing

In [4]:
# mapping function to convert string based categorical data to integer
# e.g. ABC notation to integer so that the algo can interpret it
# input sequences for the network will be developed
# followed by respective outputs
# normalizing input to avoid problems which may occur by creating new values that maintain the general distribution and ratios in the source data

In [5]:
def prepare_sequences(notes, pitchnames, n_vocab):
    
    ### MAP FUNCTION ###
    # create a dictionary to map notes to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    # length of each sequence = 100
    # this means that the previous 100 notes are used to predict the next note
    
    network_input = []
    output = []
    
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

### Creating Model Network and Importing Weights

In [6]:
def create_network(network_input, n_vocab):
    
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('poke-weights.hdf5')

    return model

### Note Generation

In [7]:
def generate_notes(model, network_input, pitchnames, n_vocab):

    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    ### MAP FUNCTION ###
    # create a dictionary to map integers to notes
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # network input is based on the sequence of notes from previous function
    pattern = network_input[start]
    # the notes generated based on the sequences defined in the previous function stored in an array
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

### MIDI File Generation

In [27]:
def create_midi(prediction_output):
    
    # offset = the position of the read/write pointer within the file
    offset = 0
    # generated notes placed into an array
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='midi_output/project_output.mid')
    
if __name__ == '__main__':
    generate()